/* Welcome to the SQL mini project. You will carry out this project partly in
the PHPMyAdmin interface, and partly in Jupyter via a Python connection.

This is Tier 1 of the case study, which means that there'll be more guidance for you about how to 
setup your local SQLite connection in PART 2 of the case study. 

The questions in the case study are exactly the same as with Tier 2. 

PART 1: PHPMyAdmin
You will complete questions 1-9 below in the PHPMyAdmin interface. 
Log in by pasting the following URL into your browser, and
using the following Username and Password:

URL: https://sql.springboard.com/
Username: student
Password: learn_sql@springboard

The data you need is in the "country_club" database. This database
contains 3 tables:
    i) the "Bookings" table,
    ii) the "Facilities" table, and
    iii) the "Members" table.

In this case study, you'll be asked a series of questions. You can
solve them using the platform, but for the final deliverable,
paste the code for each solution into this script, and upload it
to your GitHub.

Before starting with the questions, feel free to take your time,
exploring the data, and getting acquainted with the 3 tables. */



In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import LocalSQLConnection as lsc

# Question 1 

/* Q1: Some of the facilities charge a fee to members, but some do not.
Write a SQL query to produce a list of the names of the facilities that do. */


In [3]:
q = 'SELECT name FROM facilities WHERE membercost != 0'
df = lsc.query_database(q)
df

,name
0,Tennis Court 1
1,Tennis Court 2
2,Massage Room 1
3,Massage Room 2
4,Squash Court


# Question 2

In [4]:
q = 'SELECT count(name) FROM facilities WHERE membercost != 0'
df = lsc.query_database(q)
print(f"there are {df.iloc[0,0]} facilities that do not charge a fee. ")

there are 5 facilities that do not charge a fee. 


# Question 3

/* Q3: Write an SQL query to show a list of facilities that charge a fee to members,
where the fee is less than 20% of the facility's monthly maintenance cost.
Return the facid, facility name, member cost, and monthly maintenance of the
facilities in question. */


In [5]:
q = """
SELECT facid, name, membercost, monthlymaintenance FROM facilities 
WHERE membercost != 0
AND membercost < 0.2 * monthlymaintenance
"""
df = lsc.query_database(q)
df

,facid,name,membercost,monthlymaintenance
0,0,Tennis Court 1,5.0,200
1,1,Tennis Court 2,5.0,200
2,4,Massage Room 1,9.9,3000
3,5,Massage Room 2,9.9,3000
4,6,Squash Court,3.5,80


# Question 4

/* Q4: Write an SQL query to retrieve the details of facilities with ID 1 and 5.
Try writing the query without using the OR operator. */


In [6]:
q = 'SELECT * FROM facilities WHERE facid in (1,5)'
df = lsc.query_database(q)
df

,facid,name,membercost,guestcost,initialoutlay,monthlymaintenance
0,1,Tennis Court 2,5.0,25,8000,200
1,5,Massage Room 2,9.9,80,4000,3000


# Question 5

/* Q5: Produce a list of facilities, with each labelled as
'cheap' or 'expensive', depending on if their monthly maintenance cost is
more than $100. Return the name and monthly maintenance of the facilities
in question. */


In [7]:
q = """
SELECT name, monthlymaintenance,
    CASE WHEN monthlymaintenance >= 100 THEN 'expensive' 
         ELSE 'cheap' END as cheapness
FROM facilities         
"""
df = lsc.query_database(q)
df

,name,monthlymaintenance,cheapness
0,Tennis Court 1,200,expensive
1,Tennis Court 2,200,expensive
2,Badminton Court,50,cheap
3,Table Tennis,10,cheap
4,Massage Room 1,3000,expensive
5,Massage Room 2,3000,expensive
6,Squash Court,80,cheap
7,Snooker Table,15,cheap
8,Pool Table,15,cheap


# Question 6

/* Q6: You'd like to get the first and last name of the last member(s)
who signed up. Try not to use the LIMIT clause for your solution. */


In [8]:
q = """
SELECT firstname, surname FROM members
WHERE joindate = (SELECT MAX(joindate) from members)
"""
df = lsc.query_database(q)
df

,firstname,surname
0,Darren,Smith


# Question 7

/* Q7: Produce a list of all members who have used a tennis court.
Include in your output the name of the court, and the name of the member
formatted as a single column. Ensure no duplicate data, and order by
the member name. */


In [9]:
q = """
SELECT DISTINCT (mb.firstname || mb.surname) AS membername FROM bookings bk
LEFT JOIN facilities fc ON bk.facid = fc.facid
LEFT JOIN members mb ON bk.memid = mb.memid
WHERE fc.name IN ('Tennis Court 1', 'Tennis Court 2')
"""
df = lsc.query_database(q)
df

,membername
0,TracySmith
1,GUESTGUEST
2,TimRownam
3,DarrenSmith
4,JaniceJoplette
5,GeraldButters
6,TimBoothe
7,BurtonTracy
8,NancyDare
9,PonderStibbons


# Question 8

/* Q8: Produce a list of bookings on the day of 2012-09-14 which
will cost the member (or guest) more than $30. Remember that guests have
different costs to members (the listed costs are per half-hour 'slot'), and
the guest user's ID is always 0. Include in your output the name of the
facility, the name of the member formatted as a single column, and the cost.
Order by descending cost, and do not use any subqueries. */


In [10]:
q = """
SELECT fc.name, (mb.firstname || ' ' || mb.surname) AS name,
    CASE
        WHEN mb.memid = 0 THEN fc.guestcost
        ELSE fc.membercost
    END AS cost
FROM bookings bk
LEFT JOIN facilities fc ON fc.facid = bk.facid
LEFT JOIN members mb ON mb.memid = bk.memid
WHERE bk.starttime > '2012-09-14 00:00:00' AND bk.starttime < '2012-09-14 23:59:59'
AND cost > 30
ORDER BY cost DESC
"""
df = lsc.query_database(q)
df.head()

,name,name,cost
0,Massage Room 1,GUEST GUEST,80
1,Massage Room 1,GUEST GUEST,80
2,Massage Room 1,GUEST GUEST,80
3,Massage Room 2,GUEST GUEST,80


# Question 9

In [11]:
q = """
SELECT name, member_name, cost
FROM (
    SELECT 
        fc.name AS name, 
        (mb.firstname || ' ' || mb.surname) AS member_name,
        CASE
            WHEN mb.memid = 0 THEN fc.guestcost
            ELSE fc.membercost
        END AS cost
    FROM bookings bk
    LEFT JOIN facilities fc ON fc.facid = bk.facid
    LEFT JOIN members mb ON mb.memid = bk.memid
    WHERE bk.starttime > '2012-09-14 00:00:00' 
      AND bk.starttime < '2012-09-14 23:59:59'
) AS subquery
WHERE cost > 30
ORDER BY cost DESC;
"""
df = lsc.query_database(q)
df.head()

,name,member_name,cost
0,Massage Room 1,GUEST GUEST,80
1,Massage Room 1,GUEST GUEST,80
2,Massage Room 1,GUEST GUEST,80
3,Massage Room 2,GUEST GUEST,80


# Question 10 

/* Q10: Produce a list of facilities with a total revenue less than 1000.
The output of facility name and total revenue, sorted by revenue. Remember
that there's a different cost for guests and members! */


In [12]:
q = """
SELECT name, revenue FROM (
    SELECT fc.name, 
        SUM(CASE WHEN bk.memid = 0 THEN fc.guestcost
             ELSE fc.membercost END) AS revenue
    FROM bookings bk
    LEFT JOIN facilities fc ON fc.facid = bk.facid
    GROUP BY fc.name 
) WHERE revenue < 1000 
ORDER BY revenue
"""
df = lsc.query_database(q)
df

,name,revenue
0,Table Tennis,90.0
1,Snooker Table,115.0
2,Pool Table,265.0
3,Badminton Court,604.5


# Question 11

/* Q11: Produce a report of members and who recommended them in alphabetic surname,firstname order */


In [13]:
q = """
SELECT mb.firstname, mb.surname, rc.firstname_recommendor, rc.surname_recommendor 
FROM members mb
LEFT JOIN (
    SELECT CAST(recommendedby AS INTEGER) AS rid, firstname AS firstname_recommendor, 
        surname AS surname_recommendor 
    FROM members
    WHERE rid > 0
) rc ON rc.rid = mb.memid
ORDER BY mb.surname, mb.firstname
"""
df = lsc.query_database(q)
df

,firstname,surname,firstname_recommendor,surname_recommendor
0,Florence,Bader,Ramnaresh,Sarwin
1,Anne,Baker,None,None
2,Timothy,Baker,Joan,Coplin
3,Tim,Boothe,None,None
4,Gerald,Butters,Matthew,Genting
5,Joan,Coplin,None,None
6,Erica,Crumpet,None,None
7,Nancy,Dare,None,None
8,David,Farrell,None,None
9,Jemima,Farrell,Timothy,Baker


# Question 12

/* Q12: Find the facilities with their usage by member, but not guests */

**It is insufficiently clear what "usage" means here**

# Question 13

/* Q13: Find the facilities usage by month, but not guests */

**It is insufficiently clear what "usage" means here**